## Pandas实现模糊匹配Merge数据的方法

In [1]:
import pandas as pd
import numpy as np
import re

### 1. 两份数据

In [2]:
# 关键词数据
df_keyword = pd.DataFrame({
    "keyid": np.arange(5),
    "keyword": ["numpy", "pandas", "matplotlib", "sklearn", "tensorflow"]
})
df_keyword

,keyid,keyword
0,0,numpy
1,1,pandas
2,2,matplotlib
3,3,sklearn
4,4,tensorflow


In [3]:
# 句子数据
df_sentence = pd.DataFrame({
    "senid": np.arange(10, 17),
    "sentence": [
        "怎样用Pandas实现数据的Merge？",
        "Python之Numpy详细教程",
        "怎样使用Pandas批量拆分与合并Excel文件？",
        "怎样使用Pandas的map和apply函数？",
        "深度学习及TensorFlow简介",
        "Tensorflow和Numpy的关系",
        "基于sklearn的一些机器学习的代码"
    ]
})
df_sentence

,senid,sentence
0,10,怎样用Pandas实现数据的Merge？
1,11,Python之Numpy详细教程
2,12,怎样使用Pandas批量拆分与合并Excel文件？
3,13,怎样使用Pandas的map和apply函数？
4,14,深度学习及TensorFlow简介
5,15,Tensorflow和Numpy的关系
6,16,基于sklearn的一些机器学习的代码


### 方法1：暴力笛卡尔积 + 过滤

#### 新增数字完全一样的列

In [4]:
df_keyword["match"] = 1
df_sentence["match"] = 1

In [5]:
df_keyword

,keyid,keyword,match
0,0,numpy,1
1,1,pandas,1
2,2,matplotlib,1
3,3,sklearn,1
4,4,tensorflow,1


In [6]:
df_sentence

,senid,sentence,match
0,10,怎样用Pandas实现数据的Merge？,1
1,11,Python之Numpy详细教程,1
2,12,怎样使用Pandas批量拆分与合并Excel文件？,1
3,13,怎样使用Pandas的map和apply函数？,1
4,14,深度学习及TensorFlow简介,1
5,15,Tensorflow和Numpy的关系,1
6,16,基于sklearn的一些机器学习的代码,1


#### 实现merge

结果行数 = A表行数 * B表行数

In [7]:
df_merge = pd.merge(df_keyword, df_sentence)
df_merge

,keyid,keyword,match,senid,sentence
0,0,numpy,1,10,怎样用Pandas实现数据的Merge？
1,0,numpy,1,11,Python之Numpy详细教程
2,0,numpy,1,12,怎样使用Pandas批量拆分与合并Excel文件？
3,0,numpy,1,13,怎样使用Pandas的map和apply函数？
4,0,numpy,1,14,深度学习及TensorFlow简介
5,0,numpy,1,15,Tensorflow和Numpy的关系
6,0,numpy,1,16,基于sklearn的一些机器学习的代码
7,1,pandas,1,10,怎样用Pandas实现数据的Merge？
8,1,pandas,1,11,Python之Numpy详细教程
9,1,pandas,1,12,怎样使用Pandas批量拆分与合并Excel文件？


#### 过滤出结果

In [8]:
def match_func(row):
    return re.search(row["keyword"], row["sentence"], re.IGNORECASE) is not None

df_merge[df_merge.apply(match_func, axis=1)]

,keyid,keyword,match,senid,sentence
1,0,numpy,1,11,Python之Numpy详细教程
5,0,numpy,1,15,Tensorflow和Numpy的关系
7,1,pandas,1,10,怎样用Pandas实现数据的Merge？
9,1,pandas,1,12,怎样使用Pandas批量拆分与合并Excel文件？
10,1,pandas,1,13,怎样使用Pandas的map和apply函数？
27,3,sklearn,1,16,基于sklearn的一些机器学习的代码
32,4,tensorflow,1,14,深度学习及TensorFlow简介
33,4,tensorflow,1,15,Tensorflow和Numpy的关系


### 方法2：小表变字典做merge最后explode

#### 构建要join的key:index的关系

In [9]:
key_word_dict = {
    row.keyword : row.keyid 
    for row in df_keyword.itertuples()
}
key_word_dict

{'numpy': 0, 'pandas': 1, 'matplotlib': 2, 'sklearn': 3, 'tensorflow': 4}

#### 大表搜寻小表字典

In [10]:
def merge_func(row):
    # 新增一列，表示能匹配的keyids
    row["keyids"] = [
        keyid
        for key_word, keyid in key_word_dict.items()
        if re.search(key_word, row["sentence"], re.IGNORECASE)
    ]
    return row

df_merge = df_sentence.apply(merge_func, axis=1)

In [11]:
df_merge

,senid,sentence,match,keyids
0,10,怎样用Pandas实现数据的Merge？,1,[1]
1,11,Python之Numpy详细教程,1,[0]
2,12,怎样使用Pandas批量拆分与合并Excel文件？,1,[1]
3,13,怎样使用Pandas的map和apply函数？,1,[1]
4,14,深度学习及TensorFlow简介,1,[4]
5,15,Tensorflow和Numpy的关系,1,"[0, 4]"
6,16,基于sklearn的一些机器学习的代码,1,[3]


#### 展开然后做merge

In [12]:
df_merge.explode("keyids")

,senid,sentence,match,keyids
0,10,怎样用Pandas实现数据的Merge？,1,1
1,11,Python之Numpy详细教程,1,0
2,12,怎样使用Pandas批量拆分与合并Excel文件？,1,1
3,13,怎样使用Pandas的map和apply函数？,1,1
4,14,深度学习及TensorFlow简介,1,4
5,15,Tensorflow和Numpy的关系,1,0
5,15,Tensorflow和Numpy的关系,1,4
6,16,基于sklearn的一些机器学习的代码,1,3


In [13]:
df_result = pd.merge(
    left = df_merge.explode("keyids"),
    right = df_keyword,
    left_on = "keyids",
    right_on = "keyid"
)
df_result

,senid,sentence,match_x,keyids,keyid,keyword,match_y
0,10,怎样用Pandas实现数据的Merge？,1,1,1,pandas,1
1,12,怎样使用Pandas批量拆分与合并Excel文件？,1,1,1,pandas,1
2,13,怎样使用Pandas的map和apply函数？,1,1,1,pandas,1
3,11,Python之Numpy详细教程,1,0,0,numpy,1
4,15,Tensorflow和Numpy的关系,1,0,0,numpy,1
5,14,深度学习及TensorFlow简介,1,4,4,tensorflow,1
6,15,Tensorflow和Numpy的关系,1,4,4,tensorflow,1
7,16,基于sklearn的一些机器学习的代码,1,3,3,sklearn,1
